In [1]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function,
)
from autogen.agentchat.contrib import agent_builder
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

config_list = [
    {"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]},
]

In [2]:
task = (
    "Solve this problem: For how many positive integers $m$ does the equation ||x-1|-2|=m/100 have $4$ integer solutions?"
)
print(task)

Solve this problem: For how many positive integers $m$ does the equation ||x-1|-2|=m/100 have $4$ integer solutions?


In [9]:
admin = UserProxyAgent(
    name="Admin",
    system_message="Give the task and call on the next agent. After each agent's response, assess the given information and choose with agent is called next to contribute to solving the problem. Instruct the next agent to perform their task. Don't solve the problem.",
    description="Admin. You provide the task and decide which agent speaks next. You manage the conversation in the group chat.",
    code_execution_config=False,
    human_input_mode="NEVER",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

planner = AssistantAgent(
    name="Planner",
    system_message="Given a task, please determine the best way to solve the math problem. List the steps out. Do not solve the problem.",
    description="Planner. You list the steps needed in order to solve the math problem. You do not solve the problem, you simply list the steps.",
    llm_config={"config_list": config_list},
)

calculator = AssistantAgent(
    name="Calculator",
    llm_config={"config_list": config_list},
    system_message="Please solve the math problem based on the steps given by the planner.",
    description="Calculator. You solve the math problem using the steps provided by the planner. ",
)

checker = AssistantAgent(
    name="Checker",
    llm_config={"config_list": config_list},
    system_message="Please use the nested chats called review_chats to check the solution. If there is an error, identify the error and call the calculator in order to fix the error and get the right solution. Do not solve the problem; only state the errors. If the answer is correct, return 'TERMINATE'",
    description="Checker. You check the solution to make sure it's correct. If it's not, you prompt the needed corrections.",
)

In [10]:
steps_reviewer = AssistantAgent(
    name="steps reviewer",
    llm_config={"config_list": config_list},
    system_message="You are a reviewer responsible for checking if the planner provided the right steps in order to solve the problem. Identify any errors in the steps (and ONLY the steps), if they exist. Begin the review by stating your role."
)

solving_reviewer = AssistantAgent(
    name="steps reviewer",
    llm_config={"config_list": config_list},
    system_message="You are a reviewer responsible for checking if the calculator properly followed the steps in order to solve the problem. Identify any errors the calculator made when solving, if they exist. Begin the review by stating your role."
)

In [11]:
meta_reviewer = AssistantAgent(
    name="steps reviewer",
    llm_config={"config_list": config_list},
    system_message="You are a meta reviewer. You combine the advice of the other two reviewers and provide the final, summarized review."
)

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": steps_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": solving_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
checker.register_nested_chats(
    review_chats,
    trigger=calculator,
)

In [8]:
groupchat = GroupChat(
    agents=[admin, planner, calculator, checker],
    messages=[],
    allowed_or_disallowed_speaker_transitions={
        admin: [planner, calculator, checker],
        planner: [admin, calculator],
        calculator: [admin, checker],
        checker: [admin, calculator],
    },
    speaker_transitions_type="allowed",
)

manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

chat_history = admin.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Solve this problem: For how many positive integers $m$ does the equation ||x-1|-2|=m/100 have $4$ integer solutions?

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To solve this problem, follow these steps:

1. Start by considering the nature of the modulus function. Realize that the function's graph will "break" wherever the interior equals zero.
2. Practically, find where the function x - 1 and -(x - 1) equal zero. 
3. Similarly, find when |x - 1| - 2 and -(|x - 1| - 2) equal zero as well.
4. Use the points you found to split the function into different regions.
5. Draw the lines corresponding to each region and see that the number of intersections is directly related to the value of 'm' and the number of solutions.
6. For every region, the line y = m/100 when intersected with the graph can give 0, 1 or 2 solutions for the corresponding range of m. 
7. Try to find the rang